In [39]:
import pandas as pd
import numpy as np

data = pd.read_csv(r'D:\.Universidade\Doutoramento_Engenharia_Mecanica\Datasets\Backup_save\Dados_SISMA\Testes U500Xd2\Testes U500Xd2\pinças\ensaio5\2025-05-23_16-55-08\daq_data.csv')

data_daq = np.array(data[['Cu']]).squeeze()
print(data_daq.shape)
print(data.head())

(71700,)
           Time             Batch_ID        Cu
0  1.748016e+09  1748015779986666600 -0.328311
1  1.748016e+09  1748015779986666600  0.152759
2  1.748016e+09  1748015779986666600  0.287854
3  1.748016e+09  1748015779986666600  0.225249
4  1.748016e+09  1748015779986666600  0.044024


In [40]:
batch_rows = np.arange(0, len(data), 100)
newbatch_id = []
j = 0

for i in range(len(data)):
    # Prevent overflow
    if j >= len(batch_rows):
        newbatch_id.append(int(data["Time"].iloc[-1] * 1e9))  # Repeat last valid time
    else:
        newbatch_id.append(int(data["Time"].iloc[batch_rows[j]] * 1e9))

    # Update batch index every 100 rows
    if (i + 1) % 100 == 0:
        j += 1

newbatch_id = np.array(newbatch_id)
unique_newbatch_id = np.unique(newbatch_id)

print(len(newbatch_id))

#Compute Cu_rms
def moving_rms(signal, window_size):
    # Square the signal
    squared = np.power(signal, 2)

    # Compute moving average of squared values using 'valid' to avoid edge effects
    window = np.ones(window_size) / window_size
    mean_squared = np.convolve(squared, window, mode='valid')

    # Take square root to get RMS
    rms = np.sqrt(mean_squared)

    # Pad the beginning with zeros for the first (window_size - 1) values
    padded_rms = np.concatenate([np.zeros(window_size - 1), rms])

    return padded_rms

# Apply RMS filter
window_size = 40
signal = data_daq.astype(float)
smoothed_rms = moving_rms(signal, window_size)

print(len(smoothed_rms))

71700
71700


In [41]:
# Define measurement name and optional tag
measurement = "sensor_data"
experiment_name = "expteste"

lines = []
for i, row in data.iterrows():
    timestamp = float(row["Time"])
    Batch_=  newbatch_id[i]
    cu_value = float(row["Cu"])
    cu_rms = float(smoothed_rms[i])

    # Construct InfluxDB line
    line = f"{measurement},experiment_name={experiment_name},batch_id={Batch_} Cu={cu_value},Cu_rms={cu_rms} {timestamp}"
    lines.append(line)

# Write to file
with open("influx_simulated.txt", "w") as f:
    for line in lines:
        f.write(line + "\n")

print("Simulated InfluxDB data written to influx_simulated.txt")

Simulated InfluxDB data written to influx_simulated.txt
